In [122]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import hdbscan

In [123]:
data = pd.read_csv("train.csv")
data.head()

,ID,n0,n1,n2,n3,n4,n5,n6,n7,n8,...,n4087,n4088,n4089,n4090,n4091,n4092,n4093,n4094,n4095,category
0,0,0.0,0.000000,1.272801,0.290501,0.581446,0.000000,0.000000,0,0.000000,...,1.645888,0.869640,0.302432,0.953719,0.022545,0.498048,0.0,0.034988,0.692382,Orange_Ripe
1,1,0.0,0.000000,1.542096,0.000000,0.896557,0.049978,0.000000,0,0.117847,...,1.504220,0.622686,0.588427,0.524415,0.305426,0.386204,0.0,0.000000,0.668196,Banana_Ripe
2,2,0.0,0.000000,1.098595,0.571866,0.500355,0.000000,0.000000,0,0.493137,...,1.169341,0.913239,0.064404,0.531270,0.000000,0.471604,0.0,0.000000,0.658250,Mango_Raw
3,3,0.0,0.101666,1.159194,0.599216,0.893206,0.000000,0.200139,0,0.645675,...,0.560686,1.243676,0.432523,0.701881,0.000000,0.589985,0.0,0.000000,0.591165,Leeche_Raw
4,4,0.0,0.000000,1.178603,0.362568,0.577602,0.000000,0.000000,0,0.079862,...,1.206032,0.736831,0.345906,0.878515,0.119000,0.261441,0.0,0.000000,0.458905,Mango_Ripe


In [124]:
x = data.iloc[:, 1:-1].values
x = np.array(x, dtype=float)
y = data.iloc[:, -1].values

In [125]:
print(set(y))

{'Orange_Raw', 'Apple_Ripe', 'Strawberry_Raw', 'Orange_Ripe', 'Strawberry_Ripe', 'Papaya_Ripe', 'Banana_Ripe', 'Pomengranate_Raw', 'Papaya_Raw', 'Leeche_Raw', 'Guava_Ripe', 'Coconut_Raw', 'Apple_Raw', 'Guava_Raw', 'Banana_Raw', 'Mango_Ripe', 'Mango_Raw', 'Coconut_Ripe', 'Pomengranate_Ripe', 'Leeche_Ripe'}


In [126]:
#converting strings to numeric labels
label = 0
labels = {}
inverse_labels = {}
for i in set(y):
    labels[i] = label
    inverse_labels[label] = i
    label += 1

for i in range(len(y)):
    y[i] = labels[y[i]]

y = np.array(y, dtype=float)

print(f"Number of classes: {label}")
pd.DataFrame(y).head()

Number of classes: 20


,0
0,3.0
1,6.0
2,16.0
3,9.0
4,15.0


In [127]:
search_space = [
    Integer(20, 100, name='pca_n_components'),
    Integer(500, 1000, name='random_forest_n_estimators'),
    Integer(10, 20, name='random_forest_max_depth'),
    Integer(250, 500, name='isolation_forest_n_estimators'),
    Real(0.1, 0.5, name='isolation_forest_contamination'),
    Integer(1, 20, name='hdbscan_min_samples'),
    Integer(20, 80, name='hdbscan_min_cluster_size'),
]

In [128]:
def objective(params):
    pca_n_components, random_forest_n_estimators, random_forest_max_depth, isolation_forest_n_estimators, isolation_forest_contamination, hdbscan_min_samples, hdbscan_min_cluster_size = params[0], params[1], params[2], params[3], params[4], params[5], params[6]
    #pre-processing
    scaler = StandardScaler()
    pca = PCA(n_components=pca_n_components)
    x_processed = pca.fit_transform(scaler.fit_transform(x))

    #outlier-detection
    isolation_forest = IsolationForest(n_estimators=isolation_forest_n_estimators, contamination=isolation_forest_contamination)
    isolation_forest.fit(x_processed)
    x_transformed = x_processed[np.where(isolation_forest.predict(x_processed) != -1)[0]]
    y_transformed = y[np.where(isolation_forest.predict(x_processed) != -1)[0]]

    #clustering
    clusterer = hdbscan.HDBSCAN(min_samples=hdbscan_min_samples, min_cluster_size=hdbscan_min_cluster_size)
    cluster_labels = clusterer.fit_predict(x_transformed)
    x_clustered = np.concatenate((x_transformed, cluster_labels.reshape(-1, 1)), axis=1)

    #classification
    rf = RandomForestClassifier(n_estimators=random_forest_n_estimators, max_depth=random_forest_max_depth, random_state=1)

    #accuracy
    return -np.mean(cross_val_score(rf, x_clustered, y_transformed, cv=5, n_jobs=-1, scoring='accuracy'))

In [129]:
result = gp_minimize(objective, search_space, n_calls=50, random_state=42, verbose=10)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 6.7543
Function value obtained: -0.6682
Current minimum: -0.6682
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 4.0918
Function value obtained: -0.6259
Current minimum: -0.6682
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 10.6388
Function value obtained: -0.6814
Current minimum: -0.6814
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 5.6807
Function value obtained: -0.5889
Current minimum: -0.6814
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 9.2094
Function value obtained: -0.6755
Current minimum: -0.6814
Iteration No: 6 started.

In [130]:
print('Best hyperparameters:', result.x)
print('Best score:', -result.fun)

Best hyperparameters: [100, 730, 20, 266, 0.13184464846274235, 1, 20]
Best score: 0.70521327014218


In [131]:
x_test = pd.read_csv('test.csv').values
x_test = np.array(x_test, dtype='float')

In [140]:
#pre-processing
scaler = StandardScaler()
pca = PCA(n_components=100)
x_processed = pca.fit_transform(scaler.fit_transform(x))
x_test_processed = pca.fit_transform(scaler.fit_transform(x_test))

#outlier-detection
isolation_forest = IsolationForest(n_estimators=266, contamination=0.13184464846274235)
isolation_forest.fit(x_processed)
x_transformed = x_processed[np.where(isolation_forest.predict(x_processed) != -1)[0]]
y_transformed = y[np.where(isolation_forest.predict(x_processed) != -1)[0]]

#clustering
clusterer = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=20)
cluster_labels = clusterer.fit_predict(x_transformed)
x_clustered = np.concatenate((x_transformed, cluster_labels.reshape(-1, 1)), axis=1)

clusterer = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=20)
cluster_labels = clusterer.fit_predict(x_test_processed)
x_test_clustered = np.concatenate((x_test_processed, cluster_labels.reshape(-1, 1)), axis=1)

#classification
rf = RandomForestClassifier(n_estimators=730, max_depth=20, random_state=1)

rf.fit(x_clustered, y_transformed)

#predict labels
y_pred = rf.predict(x_test_clustered)

predicted_categories = []

for i in range(len(y_pred)):
    predicted_categories.append(inverse_labels[y_pred[i]])

ids = [int(i) for i in x_test[:, 0]]

results = pd.DataFrame({'ID': ids, 'Category': predicted_categories})
results.to_csv('submission.csv', index=False)